# COVisualize-19

## [COMS 4771](http://www.cs.columbia.edu/~verma/classes/ml/index.html) Final Project

Classifying COVID-19 patients from lung scans.

Anthony Krivonos

In [61]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Library Imports

In [62]:
import os, sys

from os import mkdir, chdir
from os.path import realpath, join, dirname, exists

import cv2
import numpy as np
import pandas as pd

### Import Data

In [63]:
"""
    Read CSVs into DataFrames.
"""

train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

test_data

,id,filename
0,0,test/img-0.jpeg
1,1,test/img-1.jpeg
2,2,test/img-2.jpeg
3,3,test/img-3.jpeg
4,4,test/img-4.jpeg
...,...,...
479,479,test/img-479.jpeg
480,480,test/img-480.jpeg
481,481,test/img-481.jpeg
482,482,test/img-482.jpeg


### Quick Settings

Keep these updated, so we only have to do certain tasks (like preprocessing) once.

In [64]:
SETTINGS = {

    # Preprocess the images at all?
    "PREPROCESS": False,
    
    # Preprocess the images using specific methods
    "PREPROCESS_METHOD_A": False,
    "PREPROCESS_METHOD_B": False,
    "PREPROCESS_METHOD_C": False,
    "PREPROCESS_METHOD_D": False,
    
}

SETTINGS

{'PREPROCESS': False,
 'PREPROCESS_METHOD_A': False,
 'PREPROCESS_METHOD_B': False,
 'PREPROCESS_METHOD_C': False,
 'PREPROCESS_METHOD_D': False}

### Image Preprocessing

#### Method A – Square images w/ padding

1. Find smallest-dimension training image dimension. This will be the standard image size for both training and testing.
2. Take this image's larger dimension and resize all images into squares with that side length. While doing this, center the images and pad the left and right sides.

#### Method B – Crop images, ignoring aspect ratio

1. Find smallest-dimension training image dimensions. This will be the standard image size for both training and testing.
2. Resizeevery image to this height and width, ignoring the aspect ratio of each image.

#### Method C – Crop images to smallest size, maintaining aspect ratio

1. Find smallest-dimension training image dimensions. This will be the standard image size for both training and testing.
2. Resize and crop every image to this height and width, maintaining the aspect ratio of each image and centering its source.

#### Method D – Crop images to square, maintaining aspect ratio

1. Find smallest-dimension training image dimension. This will be the standard image size for both training and testing.
2. Resize and crop every image to this size, maintaining the aspect ratio of each image and centering its source.

##### Tools
- Uses OpenCV (`CV2`)

In [65]:
##
#   Directory Names
##

# Name of the directory to put the processed images in
PROCESSED_DIRECTORY = 'processed'

# Different process method directories
METHOD_A_DIRECTORY = 'method_a'
METHOD_B_DIRECTORY = 'method_b'
METHOD_C_DIRECTORY = 'method_c'
METHOD_D_DIRECTORY = 'method_d'


##
#   Image Writing Helper Function
##
def relative_imwrite(relative_filepath, img):
    """
    Call cv2.imwrite(..., img) on a relative file path.
    :param relative_filepath: The path (i.e. 'processed/train/img-2.jpeg').
    :param img: The cv2 image.
    """
    
    # Store current working directory so we can navigate back
    cwd = os.getcwd()
    
    # Get index of last slash to use it as a splitting point
    split_idx = relative_filepath.rindex("/")
    
    # Create a relative path and filename from this
    relative_path = relative_filepath[:split_idx]
    file_name = relative_filepath[(split_idx + 1):]
    
    # Extract the directory names
    directory_names = relative_path.split("/")
    top_directory_name = cwd
    
    # Create the directory if it doesn't exist and then move to it
    # Repeat this for every subdirectory
    for dir_name in directory_names:
        top_directory_name = join(top_directory_name, dir_name)
        if not exists(top_directory_name):
            mkdir(dir_name)
        chdir(dir_name)

    # Save the file at the given path
    file_path = "./" + file_name
    cv2.imwrite(file_path, img)
    chdir(cwd)


##
#   Get processed training data as dataframe
##

def get_train_df(process_method_directory = METHOD_A_DIRECTORY):
    """
    Given a preprocessing directory, returns a dataframe with the training images and their labels.
    Uses caching to speed up loading.
    :param process_method_directory: The preprocessing directory to load.
    :return: The dataframe with training data and labels.
    """
    pickle_name = process_method_directory + ".pkl"
    pickle_path = "processed/" + pickle_name
    
    try:
        # Try to read the training pickle, if it exists
        train_pickle = pd.read_pickle(pickle_path)
    except:
        # The training pickle doesn't exist, so we create one and return it
        train_images = {}
        for _, row in train_data.iterrows():
            
            # Read image
            id, filename = row['id'], row['filename']
            image = cv2.imread("processed/" + process_method_directory + "/" + row['filename'], 0)
            train_images[id] = {
                "data": image,
                "label": row['label']
            }
        train_pickle = pd.DataFrame.from_dict(train_images, orient='index')
        train_pickle.to_pickle(pickle_path)
    return train_pickle


if SETTINGS["PREPROCESS"]:

    # Maps of file names to CV2 images
    train_images = {}
    test_images = {}
    
    min_train_image_width = sys.maxsize
    min_train_image_height = sys.maxsize
    
    ##
    #   Traverse train images and record smallest dimension
    ##
    
    # Find the smallest training image, storing the images as they're traversed
    for _, row in train_data.iterrows():
        
        # Read image
        id, filename = row['id'], row['filename']
        image = cv2.imread(row['filename'])
        train_images[id] = image
        
        # Record minimum height and width
        height, width, _ = image.shape
        min_train_image_width = min(min_train_image_width, width)
        min_train_image_height = min(min_train_image_height, height)
    
    # Instantiate the smallest size of the two dimensions in another variable
    min_train_image_size = min(min_train_image_width, min_train_image_height)
    
    ##
    #   Traverse test images only
    ##
    
    for _, row in test_data.iterrows():
        
        # Read image
        id, filename = row['id'], row['filename']
        image = cv2.imread(row['filename'])
        test_images[id] = image

#### Method A

In [66]:
def resize_and_pad(cv2_img, to_size, padding_color=0):
    """
    Resize the given CV2 image to a square with the given size length, and then pad it with the given color.
    Adapted from https://stackoverflow.com/questions/44720580/resize-image-canvas-to-maintain-square-aspect-ratio-in-python-opencv.
    :param cv2_img: The CV2 image obtained via cv2.imread(...).
    :param to_size: The desired size int.
    :param padding_color: A color int, list, tuple, or ndarray.
    :return: The padded image.
    """
    
    # Create height and width variables for better naming
    to_height = to_width = to_size

    # Get actual image dimensions
    height, width = cv2_img.shape[:2]
    aspect_ratio = width / height

    # Interpolate differently based on the image's relative size
    if height > to_height or width > to_width:
        # Shrink image via inter area as its too large
        interp = cv2.INTER_AREA
    else:
        # Stretch image via inter cubic as its too small
        interp = cv2.INTER_CUBIC

    is_image_horizontal = aspect_ratio > 1
    is_image_vertical = aspect_ratio < 1
    
    # Height and width we're resizing the image to
    new_height, new_width = to_height, to_width
    
    # Padding around the new image's inner edges
    pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

    if is_image_horizontal:
        # Image is horizontal, so requires vertical padding
        new_height = np.round(new_width / aspect_ratio).astype(int)
        pad_vert = (to_height - new_height) / 2
        pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)

    elif is_image_vertical:
        # Image is vertical, so required horizontal padding
        new_width = np.round(new_height * aspect_ratio).astype(int)
        pad_horiz = (to_width - new_width) / 2
        pad_left, pad_right = np.floor(pad_horiz).astype(int), np.ceil(pad_horiz).astype(int)

    # If only one color is provided and the image is RGB, then set the padding color to an array of length 3
    if len(cv2_img.shape) is 3 and not isinstance(padding_color, (list, tuple, np.ndarray)):
        padding_color = [padding_color] * 3

    # Resize the image to the newly calculated dimensions and interpolation strategy
    new_img = cv2.resize(cv2_img, (new_width, new_height), interpolation=interp)
    
    # Add the calculated borders around the image
    new_img = cv2.copyMakeBorder(new_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padding_color)

    return new_img

if SETTINGS["PREPROCESS"] and SETTINGS["PREPROCESS_METHOD_A"]:

    # Add a black border around the resized images
    COLOR = 0
    
    ##
    #   Resize Training Data
    ##
    
    # Resize and write training images
    for id in train_images.keys():
        train_image = train_images[id]
        resized_train_image = resize_and_pad(train_image, min_train_image_size, COLOR)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_A_DIRECTORY + "/train/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_train_image)
    
    ##
    #   Resize Test Data
    ##
    
    # Resize and write testing images
    for id in test_images.keys():
        test_image = test_images[id]
        resized_test_image = resize_and_pad(test_image, min_train_image_size, COLOR)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_A_DIRECTORY + "/test/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_test_image)

#### Method B

In [67]:
def resize_ignoring_aspect_ratio(cv2_img, to_width, to_height):
    """
    Resizes the image to the given size, ignoring aspect ratio.
    :param cv2_img: The image to resize.
    :param to_width: The desired width.
    :param to_height: The desired height.
    :return: A new, resized cv2 image.
    """
    return cv2.resize(cv2_img, (to_width, to_height), interpolation = cv2.INTER_AREA)

if SETTINGS["PREPROCESS"] and SETTINGS["PREPROCESS_METHOD_B"]:
    
    ##
    #   Resize Training Data
    ##
    
    # Resize and write training images
    for id in train_images.keys():
        train_image = train_images[id]
        resized_train_image = resize_ignoring_aspect_ratio(train_image, min_train_image_width, min_train_image_height)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_B_DIRECTORY + "/train/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_train_image)
    
    ##
    #   Resize Test Data
    ##
    
    # Resize and write testing images
    for id in test_images.keys():
        test_image = test_images[id]
        resized_test_image = resize_ignoring_aspect_ratio(test_image, min_train_image_width, min_train_image_height)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_B_DIRECTORY + "/test/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_test_image)
        

#### Method C

In [68]:
def resize_maintaining_aspect_ratio(cv2_img, to_width, to_height):
    """
    Crop the given cv2 image to the desired width and height, maintaining the image's original aspect ratio.
    :param cv2_img: The image to crop.
    :param to_width: The desired width.
    :param to_height: The desired height.
    :return: The new resized and cropped image.
    """
    
    # Create height and width variables for better naming
    height, width = cv2_img.shape[:2]
    aspect_ratio = width / height
    
    # Resizing
    max_side = max(to_height, to_width)
    if height < width:
        new_height = max_side
        new_width = int(aspect_ratio * new_height)
    else:
        new_width = max_side
        new_height = int(new_width / aspect_ratio)
    resized_img = cv2.resize(cv2_img, (new_width, new_height), interpolation = cv2.INTER_AREA)
        
    # Cropping
    left_padding = int((new_width - to_width) / 2)
    right_padding = int(np.ceil((new_width - to_width) / 2))
    top_padding = int((new_height - to_height) / 2)
    bottom_padding = int(np.ceil((new_height - to_height) / 2))
    cropped_img = resized_img[top_padding:(new_height - bottom_padding), left_padding:(new_width - right_padding)]
    
    return cropped_img

if SETTINGS["PREPROCESS"] and SETTINGS["PREPROCESS_METHOD_C"]:
    
    ##
    #   Resize Training Data
    ##
    
    # Resize and write training images
    for id in train_images.keys():
        train_image = train_images[id]
        resized_train_image = resize_maintaining_aspect_ratio(train_image, min_train_image_width, min_train_image_height)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_C_DIRECTORY + "/train/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_train_image)
    
    ##
    #   Resize Test Data
    ##
    
    # Resize and write testing images
    for id in test_images.keys():
        test_image = test_images[id]
        resized_test_image = resize_maintaining_aspect_ratio(test_image, min_train_image_width, min_train_image_height)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_C_DIRECTORY + "/test/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_test_image)

        

#### Method D

In [69]:
def resize_to_square_maintaining_aspect_ratio(cv2_img, to_size):
    """
    Wrapper around resize_maintaining_aspect_ratio to produce a cropped square image.
    :param cv2_img: The image to crop.
    :param to_size: The desired square side size.
    :return: The new resized and cropped image.
    """
    return resize_maintaining_aspect_ratio(cv2_img, to_size, to_size)

if SETTINGS["PREPROCESS"] and SETTINGS["PREPROCESS_METHOD_D"]:
    
    ##
    #   Resize Training Data
    ##
    
    # Resize and write training images
    for id in train_images.keys():
        train_image = train_images[id]
        resized_train_image = resize_to_square_maintaining_aspect_ratio(train_image, min_train_image_size)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_D_DIRECTORY + "/train/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_train_image)
    
    ##
    #   Resize Test Data
    ##
    
    # Resize and write testing images
    for id in test_images.keys():
        test_image = test_images[id]
        resized_test_image = resize_to_square_maintaining_aspect_ratio(test_image, min_train_image_size)
        image_dir = PROCESSED_DIRECTORY + "/" + METHOD_D_DIRECTORY + "/test/img-" + str(id) + ".jpeg"
        relative_imwrite(image_dir, resized_test_image)

### Heuristical Understanding of Lung X-Rays

Before proceeding, dozens of images of **normal**, **viral**, **bacterial**, and **covid**, infections (or lack thereof) were scanned. Then, one image from each class was analyzed.

| Type of Infection | Image File        | Image                                                           |
| ----------------- |:-----------------:| ---------------------------------------------------------------:|
| Normal            | train/img-0.jpeg  | <img src="train/img-0.jpeg" style="width:400px;height:300px" /> |
| Viral             | train/img-11.jpeg | <img src="train/img-11.jpeg" style="width:400px;height:300px" /> |
| Bacterial         | train/img-21.jpeg | <img src="train/img-21.jpeg" style="width:400px;height:300px" /> |
| COVID             | train/img-13.jpeg | <img src="train/img-13.jpeg" style="width:400px;height:300px" /> |

#### Normal Lungs

Normal lung images are usually the most contrastful and least inflammated.

#### Viral Lungs

Viral lungs seem deflated and have more strain-related noise around the center.

#### Bacterial Lungs

Bacterial lungs look very similar to viral lungs, except have more noticeable cloudiness above the lungs.

#### COVID Lungs

COVID (COVID-19) lungs have the most prominent deformation towards the bottom of the lungs.

### Validation

Our goal is to classify every image in the testing set (`test.csv`). Thus, we will be splitting the images in `train.csv` to use for both training and validation.

#### Method A – k-Fold Cross-Validation

1. Choose `k` number of folds.
2. Iterate over each fold, making that fold the **test** fold. The rest of the folds will be used for **training**.
3. Record the model accuracy as the unweighted mean of the accuracy over each iteration.
4. Compare the run of each classification algorithm using this method.

#### Method B – Leave-One-Out Cross-Validation (LOOCV)

1. Select 1 test sample.
2. Train on n - 1 samples and record the accuracy.
3. Repeat for the entire training set and record the model accuracy as the unweuighted mean of the accuracy over each iteration.
4. Compare the run of each classification algorithm using this method.


##### Every classifier will run through each of these techniques. Then,
- if a classifier has greatest accuracy for both validation methods A and B, then the most accurate classifier was found.
- if each method produces a different most accurate classifier, we will pick the classifier that produced the highest accuracy from either of these methods.

In [70]:
def k_fold_cv(classifier, train_data, k, *classifier_args):
    """
    :param classifier: Function that takes in (training data, test data, classifier_args) and returns a testing accuracy.
    :param train_data: List of training data to validate on, with labels.
    :param k: The number of folds to train on.
    :return: The mean classifier accuracy over all folds.
    """
    total_accuracy = 0
    for i in range(k):
        test_fold = np.array(train_data[i * k : (i + 1) * k])
        train_fold = np.array(train_data[0 : i * k] + train_data[(i + 1) * k :])
        accuracy = classifier(train_fold, test_fold, classifier_args)
        total_accuracy += accuracy
    mean_accuracy = 0 if total_accuracy is 0 else (total_accuracy / k)
    return mean_accuracy

def loo_cv(classifier, train_data, *classifier_args):
    """
    :param classifier: Function that takes in (training data, test data, classifier_args) and returns a testing accuracy.
                       Same as k-fold for k = 1.
    :param train_data: List of training data to validate on, with labels.
    :return: The mean classifier accuracy over all folds.
    """
    return k_fold_cv(classifier, train_data, 1, classifier_args)

### Classification

We'll now lay the pros and cons for, expectations of, and variations of each classifier that can perform classification of each lung image into
the classes $ Y \in \{ \text{normal}, \text{viral}, \text{bacterial}, \text{covid} \} $.

#### Method A – SVM with Various Kernels

##### Process
1. Flatten the image into a one-dimensional vector, each component representing the shading of the image from 0 to 1.

...

#### Method B – 10-Layer Neural Network (NN)

##### Process
1. Flatten the image into a one-dimensional vector, each component representing the shading of the image from 0 to 1.

...

#### Method C – 2-Layer Convolutional Neural Net (CNN) feeding into 10-Layer NN from Method B

##### Description
Convolutional neural networks 

##### Pros
- Multi-layer
- Faster to get started
- More fine-tuned versus regular CNN

##### Cons
- Comparatively slow
- Parameter tuning is more difficult
- Needs a big dataset

#### Method C – Pre-Trained ResNet50 (using `keras`)

##### Description
The ResNet is a pre-trained model that learns the residuals of the input layer. Like in regression, a residual is the difference between
observed and expected values of data. As evident from its name, this ResNet has 50 layers, and it has repeatedly been shown that training using
this model is easier than training conventional CNNs.

##### Pros
- Multi-layer
- Faster to get started
- More fine-tuned versus regular CNN

##### Cons
- Comparatively slow
- Parameter tuning is more difficult
- Needs a big dataset

In [71]:
LABELS = [ 'normal', 'viral', 'bacterial', 'covid' ]

#### Method A

##### Statistics

C = 0.01, epochs = 1000, eps = 0.0001: accuracy: 0.8333333333333334

In [72]:
def sigmoid(w, x):
    return 1 / (1 + np.exp(-np.dot(w, x)))

def classifier_svm(train_data, test_data, epochs = 1000, C = 0.1, ε = 0.0001):
    
    # For each label, do a one-versus-all classification and add it to the test_data frame
    for focus_label in LABELS:
        # Learn weights for the focus label
        w = learn_weights(train_data, epochs, C, ε, focus_label)

        # Create a column for the classification result for the explicit focus label
        test_data['cls_' + focus_label] = ""
        
        # Get weight * row dot products for each row
        for i, test_row in test_data.iterrows():
            x = test_row.loc['data'].flatten()
            cls_result = np.dot(w, x)
            test_row['cls_' + focus_label] = cls_result
        
    # Classify each test row
    accuracy = 0
    for i, test_row in test_data.iterrows():
        label = test_row.loc['label']
        cls = LABELS[np.argmax(test_row.iloc[2:])]
        accuracy += int(cls == label)
    
    # Calculate accuracy
    accuracy = 0 if accuracy == 0 else accuracy / test_data.shape[0]
    
    return accuracy

# Learn weights vector
def learn_weights(X, epochs, C, ε, focus_label):
    get_label = lambda l: 1 if l == focus_label else -1
    
    # Initialize weights and lambda slack variable vectors
    w_len = X.iloc[0].loc['data'].shape[0] * X.iloc[0].loc['data'].shape[1]
    w = np.full(w_len, 0.0)
    b = 0
    
    orig_ε = ε
    for _ in range(epochs):
        for index, row in X.iterrows():
            x = row.loc['data'].flatten()
            y = row.loc['label']
            
            if 1 - get_label(y) * np.dot(w, x) + b > 0:
                w -= ε * ((1 / epochs) * w - C * get_label(y) * x)
                b -= ε * (C / epochs)
            else:
                w -= ε * (w / epochs)
            
            ε = orig_ε / epochs
    return w

train_data = get_train_df(METHOD_D_DIRECTORY)

temp_train_len = int(train_data.shape[0] * (99/100))
train_temp = train_data.iloc[0:temp_train_len]
test_temp = train_data.iloc[temp_train_len:]

epochs = 1000
c = 0.01
eps = 0.0001

classifier_svm(train_temp, test_temp, epochs, c, eps)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


0.8333333333333334